In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Mi pc no aguanta apriori :<, pruebo con mlxtend
from fim import *
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules, fpmax

In [16]:
df = pd.read_csv('./data/cleaned_online_retail.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [17]:
#A lo largo del tiempo vemos dos variaciones por cada mitad del año, por eso creo que separarlo en mitad tiene sentido sin complicar las cosas demas
part1 = df[df['InvoiceDate'] < '2011-06-01']
part2 = df[df['InvoiceDate'] >= '2011-06-01']

In [18]:
transactions1 = part1.groupby('InvoiceNo')['Description'].apply(list).tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions1).transform(transactions1)
oneHot1 = pd.DataFrame(te_ary, columns=te.columns_)

freq1 = fpgrowth(oneHot1, min_support=0.01, use_colnames=True)

In [19]:
transactions2 = part2.groupby('InvoiceNo')['Description'].apply(list).tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions2).transform(transactions2)
oneHot2 = pd.DataFrame(te_ary, columns=te.columns_)

freq2 = fpgrowth(oneHot2, min_support=0.01, use_colnames=True)

In [20]:
rules = association_rules(freq1, metric="confidence", min_threshold=0.2)
rules = rules[rules['lift'] >= 1]

confidence = association_rules(freq1, metric="confidence", min_threshold=0.2)

maximal = fpmax(oneHot1, min_support=0.01, use_colnames=True)